In [1]:
from functools import reduce
import os
from operator import or_, and_

import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State
from dateutil.parser import parse
import django
from django.db.models import Q
from flask_caching import Cache
#from jupyter_plotly_dash import JupyterDash
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from toolz import merge, isiterable

# to make jupyter-plotly-dash work, had to install jupyter_server_proxy explicitly
# pip install jupyter_server_proxy
# jupyter serverextension enable jupyter_server_proxy
# but honestly it sort of isn't very good anyway

os.environ.setdefault("DJANGO_SETTINGS_MODULE", "mastspec.settings")
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

from plotter.models import *
# from plotter.views import *
# from plotter.forms import *
from utils import rows, columns

In [2]:
app = dash.Dash()
CACHE_CONFIG = {
    'CACHE_TYPE': 'filesystem',
    'CACHE_DIR': './.cache'
}
cache = Cache()
cache.init_app(app.server, config=CACHE_CONFIG)

# we're using this to create thread-safe global values.
# also caching relatively expensive things like database lookups to the filesystem.
# is this actually slower? maybe. if so we will supplement with storing
# data in divs.

def cset(key, value):
    cache.set(key, value)

def cget(key):
    return cache.get(key)

In [3]:
def main_graph():
    return dcc.Graph(
            id = 'main-graph',
            figure = go.Figure(),
            style={'height':'100vh'}
        )

In [4]:
@cache.memoize()
def qlist(queryset, attribute):
    return list(queryset.values_list(attribute, flat=True))

@cache.memoize()
def qset_axes(queryset, x_val, y_val):
    """
    make a pair of flat lists from a queryset based on passed fields.
    for stuff like plots
    this can be made much fancier
    """
    return [qlist(queryset, x_val), qlist(queryset, y_val)]

In [5]:
def scatter(queryset, x_val, y_val):
    fig = go.Figure()
    axes = qset_axes(queryset, x_val, y_val)
    fig.add_trace(go.Scattergl(
        x = axes[0],
        y = axes[1],
        # change this to be hella popup text
        text = None,
        marker = {
            'color':'blue'
        },
        ))
    return fig

In [6]:
@cache.memoize()
def flexible_query(queryset, field, value):
    """
    little search function that checks exact and loose phrases
    """
    # allow exact phrase searches
    query = field + "__iexact"
    if queryset.filter(**{query: value}):
        return queryset.filter(**{query: value})
    # otherwise treat multiple words as an 'or' search
    query = field + "__icontains"
    filters = [
        queryset.filter(**{query: word}) for word in value.split(" ")
    ]
    return reduce(or_, filters)

In [7]:
@cache.memoize()
def particular_fields_search(model, search_dict, searchable_fields):
    """
    'search specific defined fields' function.
    works only on strings atm!
    other things for numeric / date / etc. fields preferably
    
    'searchable fields' is a little clumsy -- but it specifically
    makes the function a little more fault-tolerant.
    """
    queryset = model.objects.all()
    # allow either single entries or lists of entries
    print(search_dict)
    for field in searchable_fields:
        entry = search_dict.get(field)
        if entry:
            entry = list(entry)
            filters = [flexible_query(queryset, field, value) for value in entry]
            queryset = reduce(or_,filters)
    return queryset

In [8]:
def filter_drop(model, element_id):
    """dropdown for filter selection"""
    return dcc.Dropdown(
        id=element_id,
        options=[{'label': filt, 'value': filt}
                 for filt in model.filters],
        style = {'width':'10rem', 'display':'inline-block'}
    )

In [9]:
def field_drop(fields, element_id):
    """dropdown for field selection -- no special logic atm"""
    return dcc.Dropdown(
        id=element_id,
        options = [{'label': field, 'value': field}
                 for field in fields]
    )

In [10]:
def field_values(queryset, field):
    """
    generates dict if all unique values in model's field
    + any and blank, for passing to HTML select constructors
    as this is based on current queryset,
    it will by default display options as constrained by other search
    parameters. this has upsides and downsides.
    it will also lead to odd behavior if care is not given.
    maybe it's a bad idea.
    """
    options_list = [
            {'label': item, 'value': item}
            for item in set(qlist(queryset,field))
            if item not in ['','nan']
        ]
    special_options = [
        {'label':'any',  'value':'any'},
        # too annoying to detect all 'blank' things atm
        # {'label':'no assigned value','value':''}
    ]
    return special_options + options_list


def model_options_drop(queryset, field, element_id):
    """
    dropdown for selecting search value for a specific field
    could end up getting unmanageable as a UI element
    """
    return dcc.Dropdown(
        id=element_id,
        options=field_values(queryset, field),
        multi = True
    )

In [11]:
@cache.memoize()
def handle_search(model, search_dict, searchable_fields):
    """
    dispatcher. right now just handles 'no assigned'
    and 'any' cases
    """
    for field, value in search_dict.items():
        if 'any' in value:
            return model.objects.all()
    return particular_fields_search(model, search_dict, searchable_fields)


In [16]:
#app = JupyterDash('TestGraph')
fig = main_graph()
#dataset_function = make_mspec_axes
graph_function = scatter
model = MSpec 
searchable_fields = ['group','formation','member']
# active queryset is explicitly stored in global cache
cset('queryset', model.objects.all())

# it's a big open UX question how to gracefully mutate filters between instruments.
# it might be that flexible band values are actually better than explicit filter
# selection.
app.layout = html.Div(children = [
    filter_drop(model,'filter-1'),
    filter_drop(model,'filter-2'),
    filter_drop(model,'filter-3'),
    # will probably want to split divs off here so we can generate new search rows
    # how do we manage that in callbacks? is there a name selector?
    field_drop(searchable_fields, 'field-search'),
    model_options_drop(cget('queryset'), 'group', 'value-search'),
    html.Button(id='submit-search', n_clicks=0, children='Submit'),
    html.Div(children = [fig]),
    html.Div(id = 'update-signal', style={"display":"none"}),
])

@app.callback(
    dash.dependencies.Output('main-graph', 'figure'),
    [dash.dependencies.Input('filter-1', 'value'),
     dash.dependencies.Input('filter-2', 'value'),
     dash.dependencies.Input('update-signal', 'value'),
    ])
def update_graph_calculation(filt, filt2):
    # move axes construction to a different function
    queryset = cget("queryset")
    return graph_function(queryset, filt+'_mean', filt2+'_mean')


@app.callback(Output('value-search', 'options'),
              [Input('field-search', 'value')],
             )
def update_model_field(field):
    """populate set of values when field in search box changes"""
    queryset = cget("queryset")
    return field_values(queryset, field)

    
@app.callback(Output('update-signal', 'value'),
              [Input('submit-search', 'n_clicks')],
              [State('field-search', 'value'),
               State('value-search', 'value')])
def update_queryset(n_clicks, field, value):
    """
    updates the spectra displayed in the graph view.
    """
    print(field,value)
    if handle_search(
        model, {field:value}, searchable_fields
    ) != cget("queryset"):
        cset(
            "queryset", 
            particular_fields_search(model, {field:value}, [field])
        )
        return n_clicks
    
    

In [13]:
app.run_server()

Dash is running on http://127.0.0.1:8050/

 in production, use a production WSGI server like gunicorn instead.

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [27/Jun/2020 00:24:42] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [27/Jun/2020 00:24:43] "GET /_dash-component-suites/dash_renderer/react@16.v1_5_0m1592852348.13.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [27/Jun/2020 00:24:43] "GET /_dash-component-suites/dash_renderer/prop-types@15.v1_5_0m1592852348.7.2.min.js HTTP/1.1" 200 -
127.0.0.1 - - [27/Jun/2020 00:24:43] "GET /_dash-component-suites/dash_renderer/polyfill@7.v1_5_0m1592852348.8.7.min.js HTTP/1.1" 200 -
127.0.0.1 - - [27/Jun/2020 00:24:43] "GET /_dash-component-suites/dash_html_components/dash_html_components.v1_0_3m1592852348.min.js HTTP/1.1" 200 -
127.0.0.1 - - [27/Jun/2020 00:24:43] "GET /_dash-component-suites/dash_core_components/dash_core_components-shared.v1_10_1m1592852348.js HTTP/1.1" 200 -
127.0.0.1 - - [27/Jun/2020 00:24:43] "GET /_dash-component-suites/dash_renderer/react-dom@16.v1_5_0m1592852348.13.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [27/Jun/2020 00

None None
Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/home/michael/.local/lib/python3.8/site-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/home/michael/.local/lib/python3.8/site-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/home/michael/.local/lib/python3.8/site-packages/flask/app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/home/michael/.local/lib/python3.8/site-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/home/michael/.local/lib/python3.8/site-packages/flask/app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "/home/michael/.local/lib/python3.8/site-packages/flask/app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "/home/michael/.local/lib/python3.8/site-packages/dash

127.0.0.1 - - [27/Jun/2020 00:24:43] "POST /_dash-update-component HTTP/1.1" 500 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/home/michael/.local/lib/python3.8/site-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/home/michael/.local/lib/python3.8/site-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/home/michael/.local/lib/python3.8/site-packages/flask/app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/home/michael/.local/lib/python3.8/site-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/home/michael/.local/lib/python3.8/site-packages/flask/app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "/home/michael/.local/lib/python3.8/site-packages/flask/app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "/home/michael/.local/lib/python3.8/site-packages/dash/dash.py",

...

127.0.0.1 - - [27/Jun/2020 00:25:38] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [27/Jun/2020 00:25:44] "POST /_dash-update-component HTTP/1.1" 200 -


member ['Blunts Point']
{'member': ['Blunts Point']}
{'member': ['Blunts Point']}


127.0.0.1 - - [27/Jun/2020 00:25:46] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [27/Jun/2020 00:25:47] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [27/Jun/2020 00:25:53] "POST /_dash-update-component HTTP/1.1" 200 -


member ['Pettegrove Point']


127.0.0.1 - - [27/Jun/2020 00:25:55] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [27/Jun/2020 00:25:56] "POST /_dash-update-component HTTP/1.1" 200 -


In [14]:
# ok so something that wraps querysets for plotly?
# we want both the requested attributes and at least the id,
# name, etc.
# so that people can query more information about chosen spectra

In [15]:
qlist(MSpec.objects.all(), 'L2_mean')

[0.0237608,
 0.0280853,
 0.0337442,
 0.0383073,
 0.0387425,
 0.0411567,
 0.0412882,
 0.0419614,
 0.0423339,
 0.042713,
 0.0441043,
 0.04674980000000001,
 0.047032300000000006,
 0.0475577,
 0.0480634,
 0.0484816,
 0.050754,
 0.0526092,
 0.053677300000000004,
 0.0546345,
 0.0552187,
 0.0559943,
 0.056528499999999995,
 0.0566413,
 0.0571621,
 0.0578167,
 0.0581981,
 0.0585279,
 0.0586969,
 0.0587613,
 0.0593444,
 0.059511,
 0.060364,
 0.0612956,
 0.0637921,
 0.0651862,
 0.0662063,
 0.0663451,
 0.0664366,
 0.0664608,
 0.0668331,
 0.0675409,
 0.0698208,
 0.0703875,
 0.0705468,
 0.0706024,
 0.0717811,
 0.0717839,
 0.07209700000000001,
 0.0720973,
 0.0721146,
 0.0724487,
 0.074193,
 0.0746481,
 0.0753126,
 0.0757304,
 0.0758824,
 0.0770826,
 0.0783989,
 0.0787573,
 0.0797769,
 0.0818121,
 0.0819947,
 0.0825772,
 0.0834599,
 0.0835757,
 0.0838774,
 0.0840248,
 0.084342,
 0.0855597,
 0.085898,
 0.0867631,
 0.0868865,
 0.0870892,
 0.089253,
 0.0900245,
 0.0934586,
 0.0939735,
 0.0941657,
 0.0942